In [23]:
import os

from matplotlib.pyplot import imshow
from PIL import Image
from pathlib import Path
import numpy as np
import colour
from dataclasses import dataclass, field
from enum import Enum
import rawpy


from colour_demosaicing import (
    EXAMPLES_RESOURCES_DIRECTORY,
    demosaicing_CFA_Bayer_bilinear,
    demosaicing_CFA_Bayer_Malvar2004,
    demosaicing_CFA_Bayer_Menon2007,
    masks_CFA_Bayer)




In [90]:
colour.plotting.colour_style()

META_KEY = b'META\x00\xa5\xa5\xa5'
RAW_KEY = b'RAW0\x000\x00\xa5'
TEST_FILES = Path('test_files_wb')
MODEL = {
    'e22': "Emotion 22"
}

norm2 = lambda x: np.clip( (x - x.max()) * (1 / (x.min() - x.max())), 0, 1)
norm = lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))
gle = lambda b: int.from_bytes(b,  byteorder='little')
gli = lambda b, s : int.from_bytes(b[s:s+4],  byteorder='little')
gls = lambda b, s : int.from_bytes(b[s:s+2],  byteorder='little')


class WhiteBalance(Enum):
    Manual = 7
    Flash = 0
    Neon = 1
    Tungsten = 2
    Shadow = 3
    Sun = 4
    Cloudy = 5
    
    def __str__(self):
        return str(self.name)
    

    
def gwad(raw, start, offset):
    return raw[start:start+offset]


def read_raw(path):
    with open(path, 'rb') as rawfp:
        raw = rawfp.read()
    return raw


def read_pwad_lumps(raw):
    lumps = {}
    ftype, num_file, offset = get_pwad_info(raw)
    for i in range(num_file):
        file_entry_start = offset+i*16
        fe_offset = gle(raw[file_entry_start:file_entry_start+4])
        fe_size = gle(raw[file_entry_start+4:file_entry_start+8])
        name = raw[file_entry_start+8:file_entry_start+16]
        #print(f'{name}: fe_offset: {fe_offset}, fe_size: {fe_size} bytes')
        lumps[name] = (fe_offset, fe_size)
    return lumps


def get_pwad_info(raw):
    ftype = raw[0:4]
    num_file = gle(raw[4:8])
    offset = gle(raw[8:12])
    #print(f'ftype: {ftype}, {num_file}, {offset}')
    return ftype, num_file, offset


@dataclass
class SinarIA:
    shutter_count: int
    camera: str
    measured_shutter_us: int
    req_shutter_us: int
    f_stop: int
    black_ref: str
    iso: int
    serial: str
    white_balance_name: WhiteBalance 
    focal_length: int
    filename: Path = field(default=Path(''))
    raw_data: bytes = field(default_factory=bytes, repr=False)
    meta: bytes = field(default_factory=bytes, repr=False)


def process_meta(meta: bytes):
    shutter_count = gli(meta, 4)
    camera = meta[20:64].decode('ascii').rstrip('\x00')
    white_balance_name = WhiteBalance(gls(meta, 100))
    shutter_time_us = gli(meta, 104)
    black_ref = meta[108:108+128].decode('ascii').rstrip('\x00')
    iso = gli(meta, 252)
    serial = meta[272:272+16].decode('ascii').rstrip('\x00')
    shutter_time_us_2 = gli(meta, 344)
    f_stop = round(gls(meta, 352)/256, 1)
    focal_length = round(gli(meta, 356)/1000, 0)
    return SinarIA(
        shutter_count=shutter_count,
        camera=camera,
        measured_shutter_us=shutter_time_us,
        req_shutter_us=shutter_time_us_2,
        f_stop=f_stop,
        black_ref=black_ref,
        iso=iso,
        serial=serial,
        meta=meta,
        white_balance_name=white_balance_name,
        focal_length=focal_length
    )        

def read_sinar(path: Path):
    raw = read_raw(path)
    lumps = read_pwad_lumps(raw)
    meta = gwad(raw, *lumps[META_KEY])
    sinar_ia = process_meta(meta)
    raw_data = gwad(raw, *lumps[RAW_KEY])
    sinar_ia.raw_data=raw_data
    sinar_ia.filename = path
    return sinar_ia
    

def read_black_ref(path: Path, img, mode='BLACK1'):
    black = read_raw(path)
    lumps = read_pwad_lumps(black)
    black0 = gwad(black, *lumps[b'BLACK0\x00\xa5'])
    black1 = gwad(black, *lumps[b'BLACK1\x00\xa5'])
    black0_img = np.asarray(Image.frombytes('I;16L', img.size, black0, 'raw'))
    black1_img = np.asarray(Image.frombytes('I;16L', img.size, black1, 'raw'))
    if mode == 'BLACK1':
        return black1_img
    elif mode == 'BLACK0': 
        return black0_img
    elif mode == 'COMBO':
        return black1_img-black0_img
    else:
        raise Exception('Mode not found!')
        
        
def get_raw_pillow(RAW: SinarIA, h, w):
    RAW0 = RAW.raw_data
    assert (h*w*16)/8 == len(RAW0)
    img = Image.frombytes('I;16L', (w,h), RAW0, 'raw')
    return img


def apply_local_black_ref(img: Image, black_path: Path, mode='COMBO'):
    BLACK = read_black_ref(black_path, img, mode=mode)
    nd_img = np.asarray(BLACK)
    return img-nd_img

def process_raw(RAW: SinarIA, h=5344, w=4008, mode='COMBO'):
    img = get_raw_pillow(RAW, h, w)
    black_path = RAW.filename.parent.absolute() / Path(RAW.black_ref).name
    nd_img = apply_local_black_ref(img, black_path, mode=mode)
    return nd_img

def sinar_ia_to_pil(path: Path, h=5344, w=4008):
    sinar_ia = read_sinar(path)
    nd_img = process_raw(sinar_ia, h=h, w=w)
    out = demosaicing_CFA_Bayer_Menon2007(norm(nd_img), "RGGB")
    return Image.fromarray((out*65536).astype('uint16'), mode='RGB')

In [91]:
raws = {}
for ia in TEST_FILES.glob('*.IA'):
    print(f'Reading {ia.name}...')
    raw = read_sinar(ia)
    raws[raw.filename.name] = raw

Reading 683003CA.IA...
Reading 68300273.IA...
Reading 6830013F.IA...
Reading 6830007F.IA...
Reading 683000EE.IA...
Reading 68300195.IA...
Reading 683002E1.IA...
Reading 683001DE.IA...
Reading 682FFF96.IA...
Reading 683004E0.IA...
Reading 6830054F.IA...


In [92]:
sorted(raws.items(), key=lambda i: i[1].shutter_count)

[('682FFF96.IA',
  SinarIA(shutter_count=2561, camera='Sinar Hy6', measured_shutter_us=4008000, req_shutter_us=4000000, f_stop=8.0, black_ref='/002000E8.EMO/682FFFAB.BR', iso=50, serial='e22-05-0311', white_balance_name=<WhiteBalance.Manual: 7>, focal_length=0.0, filename=PosixPath('test_files_wb/682FFF96.IA'))),
 ('6830007F.IA',
  SinarIA(shutter_count=2562, camera='Sinar Hy6', measured_shutter_us=4007750, req_shutter_us=4000000, f_stop=8.0, black_ref='/002000E8.EMO/68300094.BR', iso=50, serial='e22-05-0311', white_balance_name=<WhiteBalance.Flash: 0>, focal_length=0.0, filename=PosixPath('test_files_wb/6830007F.IA'))),
 ('683000EE.IA',
  SinarIA(shutter_count=2563, camera='Sinar Hy6', measured_shutter_us=4007750, req_shutter_us=4000000, f_stop=8.0, black_ref='/002000E8.EMO/68300104.BR', iso=50, serial='e22-05-0311', white_balance_name=<WhiteBalance.Neon: 1>, focal_length=0.0, filename=PosixPath('test_files_wb/683000EE.IA'))),
 ('6830013F.IA',
  SinarIA(shutter_count=2564, camera='Sin

In [93]:
test_img = Path('/Users/mgolub2/Pictures/002000E8.EMO/682FD801.IA')
sinar_ia = read_sinar(test_img)
img = process_raw(sinar_ia)

In [94]:
nimg = np.clip(norm(img), 0, 1)
nimg[np.isnan(nimg)] = 0
out = demosaicing_CFA_Bayer_Menon2007(nimg, "RGGB")
Image.fromarray((out*255).astype('uint8'), mode='RGB').rotate(90).show()



In [95]:
with open('/Users/mgolub2/Projects/sinar_emotion/ccd.dat/00525025.ref', 'rb') as ccdfp:
    ccd = ccdfp.read()
with open('/Users/mgolub2/Projects/sinar_emotion/ccd.dat/00525025.dat', 'rb') as datfp:
    dat = datfp.read()

In [96]:
from pidng.core import RAW2DNG, DNGTags, Tag
from pidng.defs import PhotometricInterpretation, Orientation,\
    CFAPattern, CalibrationIlluminant, DNGVersion, PreviewColorSpace
img.shape

(5344, 4008)

In [98]:
# uncalibrated color matrix, just for demo.
mult = 1000000000
ccm1 = [[690277635, mult],  [81520691, mult], [-52482637, mult],
        [-674076304, mult], [1469691720, mult], [769255116, mult],
        [-180923460, mult], [390943643, mult], [1601514930, mult]]

def create_ia_dng(img: SinarIA, bpp=16, mode='COMBO'):
    t = DNGTags()
    nd_img = process_raw(img, mode=mode)
    h, w = nd_img.shape
    db_model = img.serial.split('-')[0]
    t.set(Tag.ImageWidth, w)
    t.set(Tag.ImageLength, h)
    # t.set(Tag.TileWidth, w)
    # t.set(Tag.TileLength, h)
    t.set(Tag.Orientation, 8) # Rotate 270 CW
    t.set(Tag.PhotometricInterpretation, PhotometricInterpretation.Color_Filter_Array)
    t.set(Tag.SamplesPerPixel, 1)
    t.set(Tag.BitsPerSample, bpp)
    t.set(Tag.CFARepeatPatternDim, [2,2])
    t.set(Tag.CFAPattern, CFAPattern.RGGB)
    #t.set(Tag.BlackLevel, (4096 >> (16 - bpp)))
    t.set(Tag.WhiteLevel, 2**bpp-1)
    t.set(Tag.ColorMatrix1, ccm1)
    t.set(Tag.CalibrationIlluminant1, CalibrationIlluminant.D50)
    t.set(Tag.AsShotNeutral, [[1,1],[1,1],[1,1]])
    t.set(Tag.Make, img.camera)
    t.set(Tag.Model, db_model)
    t.set(Tag.DNGVersion, DNGVersion.V1_4)
    t.set(Tag.DNGBackwardVersion, DNGVersion.V1_2)
    t.set(Tag.EXIFPhotoBodySerialNumber, img.serial)
    t.set(Tag.CameraSerialNumber, img.serial)
    t.set(Tag.ExposureTime, [(img.measured_shutter_us, 1000000)])
    t.set(Tag.PhotographicSensitivity, img.iso)
    t.set(Tag.SensitivityType, 3)
    t.set(Tag.FocalLength, [(int(img.focal_length), 1)])
    t.set(Tag.UniqueCameraModel, f'{db_model} on {img.camera}')
    t.set(Tag.FNumber, [(int(img.f_stop*100), 100)])
    t.set(Tag.BayerGreenSplit, 0)
    t.set(Tag.Software, 'PYEmotionDNG')
    #t.set(Tag.PreviewColorSpace, PreviewColorSpace.sRGB)
    r = RAW2DNG()
    r.options(t, path="", compress=False)
    filename = f"{img.filename.stem}_{mode.lower()}.dng"
    r.convert(nd_img, filename=filename)
    print(f'Converted {img.filename} to {filename}')

create_ia_dng(sinar_ia)

Converted /Users/mgolub2/Pictures/002000E8.EMO/682FD801.IA to 682FD801_combo.dng


In [ ]:
for k, v in sorted(raws.items(), key=lambda i: i[1].shutter_count):
    #start = 360
    #end = 200
    with open(f'{v.shutter_count}_meta.bin', 'wb')as metafp:
        metafp.write(v.meta)
    #print(v.meta[start:start+end], end='\t')
    #print(gle(v.meta[start:start+end]))

In [13]:
img.shape



(5344, 4008)